<a href="https://colab.research.google.com/github/sthalles/SimCLR/blob/simclr-refactor/feature_eval/mini_batch_logistic_regression_evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import sys
import numpy as np
import os
import yaml
import matplotlib.pyplot as plt
import torchvision

In [ ]:
%pip install gdown

In [ ]:
def get_file_id_by_model(folder_name):
  file_id = {'resnet18_100-epochs_stl10': '14_nH2FkyKbt61cieQDiSbBVNP8-gtwgF',
             'resnet18_100-epochs_cifar10': '1lc2aoVtrAetGn0PnTkOyFzPCIucOJq7C',
             'resnet50_50-epochs_stl10': '1ByTKAUsdm_X7tLcii6oAEl5qFRqRMZSu'}
  return file_id.get(folder_name, "Model not found.")

In [ ]:
folder_name = 'resnet50_50-epochs_stl10'
file_id = get_file_id_by_model(folder_name)
print(folder_name, file_id)

In [ ]:
# download and extract model files
# os.system('gdown https://drive.google.com/uc?id={}'.format(file_id))
# os.system('unzip {}'.format(folder_name))
# !ls

In [ ]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

In [ ]:
def get_stl10_data_loaders(download, shuffle=False, batch_size=256):
  train_dataset = datasets.STL10('./data', split='train', download=download,
                                  transform=transforms.ToTensor())

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=2, drop_last=False, shuffle=shuffle)
  
  test_dataset = datasets.STL10('./data', split='test', download=download,
                                  transform=transforms.ToTensor())

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=2, drop_last=False, shuffle=shuffle)#changed num of workers
  return train_loader, test_loader

def get_cifar10_data_loaders(download, shuffle=False, batch_size=256):
  train_dataset = datasets.CIFAR10('./data', train=True, download=download,
                                  transform=transforms.ToTensor())

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=2, drop_last=False, shuffle=shuffle)
  
  test_dataset = datasets.CIFAR10('./data', train=False, download=download,
                                  transform=transforms.ToTensor())

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=2, drop_last=False, shuffle=shuffle)
  return train_loader, test_loader

In [ ]:
# with open(os.path.join('./config.yml')) as file:
#   config = yaml.load(file)

class config_class:
  arch = "resnet18"
  dataset_name = "cifar10"
  def __init__(self):
    self.arch
    self.dataset_name

config = config_class()
config.arch = "resnet18"
config.dataset_name = "cifar10"

In [ ]:
from resnet_simclr import ResNetSimCLR
# from models.resnet_simclr import  resnet_simclr
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if config.arch == 'resnet18':
  model = ResNetSimCLR(base_model='resnet18', out_dim=10).to(device)
elif config.arch == 'resnet50':
  model = torchvision.models.resnet50(pretrained=False, num_classes=10).to(device)

In [ ]:
# checkpoint = torch.load('checkpoint_0040.pth.tar', map_location=device)
# state_dict = checkpoint['state_dict']

# for k in list(state_dict.keys()):

#   if k.startswith('backbone.'):
#     if k.startswith('backbone') and not k.startswith('backbone.fc'):
#       # remove prefix
#       state_dict[k[len("backbone."):]] = state_dict[k]
#   del state_dict[k]

In [ ]:
# log = model.load_state_dict(state_dict, strict=False)
# assert log.missing_keys == ['fc.weight', 'fc.bias']

In [ ]:
if config.dataset_name == 'cifar10':
  train_loader, test_loader = get_cifar10_data_loaders(download=True)
elif config.dataset_name == 'stl10':
  train_loader, test_loader = get_stl10_data_loaders(download=True)
print("Dataset:", config.dataset_name)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, weight_decay=0.0008)
criterion = torch.nn.CrossEntropyLoss().to(device)

In [ ]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [ ]:
#visualize the dataset

import torch
from torchvision import models
from torchsummary import summary
import torchvision.models as models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
summary(model, (3, 32, 32))

In [ ]:
epochs = 100
test_accuracy_list1 = []
for epoch in range(epochs):
  print("Started epoch {}".format(epoch))
  top1_train_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(train_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)
    print("Batch {}".format(counter))
    print("x_batch:", x_batch.shape)
    print("y_batch:", y_batch.shape)
    

    logits = model(x_batch)
    loss = criterion(logits, y_batch)
    top1 = accuracy(logits, y_batch, topk=(1,))
    top1_train_accuracy += top1[0]

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  top1_train_accuracy /= (counter + 1)
  top1_accuracy = 0
  top5_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(test_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
  
    top1, top5 = accuracy(logits, y_batch, topk=(1,5))
    top1_accuracy += top1[0]
    top5_accuracy += top5[0]
  
  top1_accuracy /= (counter + 1)
  top5_accuracy /= (counter + 1)
  print(f"Epoch {epoch}\tTop1 Train accuracy {top1_train_accuracy.item()}\tTop1 Test accuracy: {top1_accuracy.item()}\tTop5 test acc: {top5_accuracy.item()}")
   #calculate the accuracy of the model on the test set
  test_accuracy_list1.append(top1_accuracy.item())
  print("Test accuracy: {}".format(test_accuracy_list1[-1]))
#plot the accuracy of the model over the epochs
#epochs in axis x
epochs1 = np.arange(1, epochs+1)
#accuracy in axis y
accuracy1 = test_accuracy_list1
plt.plot(epochs1, accuracy1, label='accuracy')
plt.legend()
plt.show()

In [ ]:
#visualize the dataset

import torch
from torchvision import models
from torchsummary import summary
import torchvision.models as models
#show what the model is made of we can compare it to the original resnet 18
# resnet18 = models.resnet18()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
summary(model, (3, 32, 32))

In [ ]:
# from torchviz import make_dot
# y = model(img)
# make_dot(y, params=dict(list(auto_model.named_parameters()))).render("torchviz", format="png")

In [ ]:
#compute model loss



In [ ]:
#copy and pasted because didn't manage to import it
from torch import nn

class LinearClassifier(nn.Module):
    def __init__(self):
        super(LinearClassifier, self).__init__()
        self.fc1 = nn.Linear(128, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
#joined model 
class JoinedModel(nn.Module):
    def __init__(self,num_classes=10):
        
        super(JoinedModel, self).__init__()
        #uses the resnets weights already trained
        self.simCLR = model
        #classifier parts
        self.classifier = LinearClassifier()
    def forward(self, x):
        x = self.simCLR.forward(x,no_projection_head=True)
        #classifier part
        x = self.classifier(x)
        return x
joined_model = JoinedModel().to(device)

In [ ]:
#verify the model shape
summary(joined_model, (3, 32, 32))

In [ ]:
#freeze all weights 
for param in joined_model.parameters():
    param.requires_grad = False
#unfreeze the classifier
joined_model.classifier.fc1.weight.requires_grad = True
joined_model.classifier.fc1.bias.requires_grad = True
joined_model.classifier.fc2.weight.requires_grad = True
joined_model.classifier.fc2.bias.requires_grad = True

In [ ]:
#verify weight are frozen

for name, param in joined_model.named_parameters():
    print(name, param.requires_grad)


In [ ]:
#train the joined model
learning_rate = 1e-3
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, joined_model.parameters()), lr=learning_rate)


In [ ]:
#a function that calculates the accuracy of the model on the test set
def test_accuracy(model, test_loader):
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images).to(device)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        return 100 * correct / total

In [ ]:
#train the classification layer of joined model
epochs = 100
test_accuracy_list2 = []
for epoch in range(epochs):
    print("Started epoch {}".format(epoch))
    top1_train_accuracy = 0
    for counter, (x_batch, y_batch) in enumerate(train_loader):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        print("Batch {}".format(counter))
        print("x_batch:", x_batch.shape)
        print("y_batch:", y_batch.shape)
        
    
        logits = joined_model(x_batch)
        loss = criterion(logits, y_batch)
        top1 = accuracy(logits, y_batch, topk=(1,))
        top1_train_accuracy += top1[0]
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    top1_train_accuracy /= (counter + 1)
    top1_accuracy = 0
    top5_accuracy = 0
    for counter, (x_batch, y_batch) in enumerate(test_loader):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
    
        logits = joined_model(x_batch)
    
        top1, top5 = accuracy(logits, y_batch, topk=(1,5))
        top1_accuracy += top1[0]
        top5_accuracy += top5[0]
    
    top1_accuracy /= (counter + 1)
    top5_accuracy /= (counter + 1)
    print(f"Epoch {epoch}\tTop1 Train accuracy {top1_train_accuracy.item()}\tTop1 Test accuracy: {top1_accuracy.item()}\tTop5 test acc: {top5_accuracy.item()}")
     #calculate the accuracy of the model on the test set
    test_accuracy_list2.append(test_accuracy(joined_model, test_loader))
    print("Test accuracy: {}".format(test_accuracy_list2[-1]))
#plot the accuracy of the model over the epochs
#epochs in axis x
epochs2 = np.arange(1, epochs+1)
#accuracy in axis y
accuracy2 = test_accuracy_list2
plt.plot(epochs2, accuracy2, label='accuracy')
plt.legend()
plt.show()



In [ ]:
# evaluate the joined model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = joined_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * (correct / total)))
    # save the model
    # torch.save(joined_model.state_dict(), './joined_model_simCLR_Classif.pth')

In [4]:
if not os.path.exists('../../saved_models'):
    os.makedirs('../../saved_models')
#save the joined model
torch.save(joined_model.state_dict(), '../../saved_models/joined_model_simCLR_Classif.pth')
#save simclr
torch.save(model.state_dict(), '../../saved_models/simCLR_Classif.pth')


NameError: name 'joined_model' is not defined